In [1]:
import pandas as pd
import csv
import time
from sklearn.model_selection import train_test_split

# check version number
import imblearn
print(imblearn.__version__)

from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder

from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression



Bad key "text.kerning_factor" on line 4 in
/Users/apple/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


0.8.0


In [2]:
# load data
train = pd.read_csv("train.csv", header = None)

test = pd.read_csv("test.csv", header = None)
FEATURE_SIZE=3

print(test.info())
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2959 entries, 0 to 2958
Columns: 961 entries, 0 to 960
dtypes: float64(960), int64(1)
memory usage: 21.7 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9388 entries, 0 to 9387
Columns: 962 entries, 0 to 961
dtypes: float64(960), int64(2)
memory usage: 68.9 MB


In [3]:
# minor_size is number of classes less than 500 instances
minor_size = 0
grouped = train.groupby(train.columns[-1])
for k,group in grouped:
        if len(group.index) < 500:
            minor_size += 1
        print ("Class label: ",k,end='')
        print("  Number:",len(group.index), end='')
        print(" (%.3f%%)" %  float(len(group.index)/len(train)*100))
minor_size

Class label:  1  Number: 578 (6.157%)
Class label:  2  Number: 579 (6.167%)
Class label:  3  Number: 590 (6.285%)
Class label:  4  Number: 571 (6.082%)
Class label:  5  Number: 577 (6.146%)
Class label:  6  Number: 580 (6.178%)
Class label:  7  Number: 570 (6.072%)
Class label:  8  Number: 580 (6.178%)
Class label:  9  Number: 586 (6.242%)
Class label:  10  Number: 579 (6.167%)
Class label:  11  Number: 190 (2.024%)
Class label:  12  Number: 195 (2.077%)
Class label:  13  Number: 196 (2.088%)
Class label:  14  Number: 190 (2.024%)
Class label:  15  Number: 190 (2.024%)
Class label:  16  Number: 194 (2.066%)
Class label:  17  Number: 193 (2.056%)
Class label:  18  Number: 190 (2.024%)
Class label:  19  Number: 193 (2.056%)
Class label:  20  Number: 195 (2.077%)
Class label:  21  Number: 98 (1.044%)
Class label:  22  Number: 97 (1.033%)
Class label:  23  Number: 97 (1.033%)
Class label:  24  Number: 98 (1.044%)
Class label:  25  Number: 96 (1.023%)
Class label:  26  Number: 98 (1.044%)
C

39

## Split Data

In [4]:
# exclude sample ID (1st column)
x_train_valid = train.iloc[:,1:-1].to_numpy()
y_train_valid = train.iloc[:,-1].to_numpy()

#split a validation set for hyper-parameter tuning
split_ratio = 0.2
x_train, x_val, y_train, y_val = train_test_split(x_train_valid, y_train_valid, test_size=split_ratio, random_state=1,stratify = y_train_valid) 

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#x_train_sc = sc.fit_transform(x_train)
#x_val_sc = sc.transform(x_val)

x_test = test.iloc[:,1:].to_numpy()
y_test = test.iloc[:,-1].to_numpy()


### Apply SMOTE

In [5]:
from imblearn.over_sampling import ADASYN, BorderlineSMOTE, SMOTE

# we only oversample during training
ada = ADASYN(random_state = 42, sampling_strategy = 'minority', n_neighbors=5)
bdsmt = BorderlineSMOTE(random_state = 42)
smt = SMOTE(random_state = 42)

x_ada_train = x_train.copy()
y_ada_train = y_train.copy()

x_bdsmt_train = x_train.copy()
y_bdsmt_train = y_train.copy()

x_smt_train = x_train.copy()
y_smt_train = y_train.copy()

# ada failed to resample all minority class at once, we manually resample minority classes(less than 500 instances)
for i in range(minor_size):
    x_ada_train, y_ada_train = ada.fit_resample(x_ada_train, y_ada_train)
    
    
x_bdsmt_train, y_bdsmt_train = bdsmt.fit_resample(x_bdsmt_train, y_bdsmt_train)
x_smt_train, y_smt_train = smt.fit_resample(x_smt_train, y_smt_train)

#print("ada size: ", len(x_ada_train))
#print("bdsmt size: ", len(x_bdsmt_train))
#print("ada size: ", len(y_ada_train))
print("bdsmt size: ", len(y_bdsmt_train))
#print("smt size: ", len(y_smt_train))

bdsmt size:  22222


## Normalizing and PCA

In [14]:
from sklearn.decomposition import PCA
from sklearn import preprocessing
import numpy as np


pca = PCA(n_components=300)
#pca = PCA(n_components = 'mle')

# normalize data first, then PCA
x_ada_train_norm = preprocessing.normalize(x_ada_train)
x_bdsmt_train_norm = preprocessing.normalize(x_bdsmt_train)
x_val_norm = preprocessing.normalize(x_val)
# PCA data
x_ada_train_pca =  pca.fit_transform(x_ada_train_norm)
x_bdsmt_train_pca =  pca.fit_transform(x_bdsmt_train_norm)
x_val_pca = pca.fit_transform(x_val_norm)

print(x_ada_train_pca.shape, x_bdsmt_train_pca.shape)


(25076, 300) (22222, 300)


## Modelling

In [22]:
from sklearn import linear_model
lr=linear_model.LogisticRegression(solver='lbfgs', multi_class='multinomial', 
                                   C = 0.000001, n_jobs = -1, max_iter = 500)
lr.fit(x_bdsmt_train,y_bdsmt_train)


LogisticRegression(C=1e-06, max_iter=500, multi_class='multinomial', n_jobs=-1)

In [23]:
#C = 0.000001, max_iter = 500
pre=lr.predict(x_val)
# print(pre)
acc=lr.score(x_val,y_val)
print(acc)

0.2124600638977636


In [21]:
#C = 0.0000001, max_iter = 500
pre=lr.predict(x_val)
# print(pre)
acc=lr.score(x_val,y_val)
print(acc)

0.2103301384451544


In [24]:
from sklearn import linear_model
lr1=linear_model.LogisticRegression(solver='lbfgs', multi_class='multinomial', 
                                   C = 0.00001, n_jobs = -1, max_iter = 500)
lr1.fit(x_bdsmt_train,y_bdsmt_train)

LogisticRegression(C=1e-05, max_iter=500, multi_class='multinomial', n_jobs=-1)

In [25]:
#C = 0.00001, max_iter = 500
train_pre = lr1.predict(x_bdsmt_train)
train_acc = lr1.score(x_bdsmt_train, y_bdsmt_train)

pre1=lr1.predict(x_val)
val_acc1=lr1.score(x_val,y_val)


In [28]:
print("train_acc", train_acc)
print("val_acc1", val_acc1)

train_acc 0.7952479524795248
val_acc1 0.25292864749733757


In [17]:
#C = 0.00001
pre=lr.predict(x_val)
# print(pre)
acc=lr.score(x_val,y_val)
print(acc)

0.20820021299254526


In [15]:
#C = 0.0001
pre=lr.predict(x_val)
# print(pre)
acc=lr.score(x_val,y_val)
print(acc)

0.19382321618743345


In [12]:
#C = 10
pre=lr.predict(x_val)
# print(pre)
acc=lr.score(x_val,y_val)
print(acc)

0.19382321618743345


In [10]:
#C = 0.01
pre=lr.predict(x_val)
# print(pre)
acc=lr.score(x_val,y_val)
print(acc)

0.1954206602768903


In [7]:
#C = 0.1
pre=lr.predict(x_val)
# print(pre)
acc=lr.score(x_val,y_val)
print(acc)

0.19382321618743345


In [8]:
#C = 1
pre=lr.predict(x_val)
# print(pre)
acc=lr.score(x_val,y_val)
print(acc)


0.19329073482428116
